In [22]:
from bs4 import BeautifulSoup 
import requests
import pandas as pd
import geocoder # Let's import geocoder hahaha.



In [23]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
url_coord = 'https://cocl.us/Geospatial_data'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
soup = BeautifulSoup(page.text, 'lxml')
table = soup.find_all('table')[0]

In [24]:
# Parse and add data to a dataframe
# Use temporary list for each row.
tabletemp=[]
for row in table.find_all('tr'):
    rowtemp=[]
    for td in row.find_all('td'):
        rowtemp = rowtemp + [td.get_text()]
    tabletemp.append(rowtemp)
    headtemp=[]
for th in table.find_all('th'):
    headtemp= headtemp + [th.get_text()]
tabletemp[0]= headtemp

In [25]:
for eachrow in tabletemp:
    for i, word in enumerate(eachrow):
        eachrow[i] = word.strip('\n')

In [26]:
new_table = pd.DataFrame(tabletemp[1:],columns=tabletemp[0])

In [27]:
delete_index = new_table[new_table['Borough']=='Not assigned'].index
new_table.drop(delete_index, axis=0, inplace=True)
new_table.reset_index(drop=True, inplace=True)
neigh_na_index = new_table[new_table['Neighbourhood']=='Not assigned'].index
new_table.iloc[neigh_na_index, 2] = new_table[new_table['Neighbourhood']=='Not assigned']['Borough']
new_table=new_table.groupby(['Postcode','Borough'], as_index=False)['Neighbourhood'].agg(lambda nbh: ', '.join(nbh))

In [28]:
new_table.tail()


,Postcode,Borough,Neighbourhood
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."
102,M9W,Etobicoke,Northwest


In [29]:
new_table['Postcode'][0]


'M1B'

In [30]:
c=pd.read_csv(url_coord)

In [31]:
new_table = new_table.join(c[['Latitude','Longitude']])

In [32]:
new_table.head()


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
